In [74]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import inspect
import talib

vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [77]:
symbol = 'AAPL'
data = vbt.AlpacaData.download(symbol,
                               start='2023-3-10',
                               end='2023-3-14',
                               # limit=1000,
                               timeframe='15m').get()

#inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


AttributeError: type object 'DOMWidget' has no attribute '_ipython_display_'

FigureWidget({
    'data': [{'close': array([150.32  , 150.4   , 150.47  , 150.64  , 150.65  , 150.95  , 150.6   ,
                              150.51  , 149.7   , 149.91  , 150.28  , 150.06  , 150.6   , 150.51  ,
                              150.26  , 150.64  , 150.6498, 150.55  , 151.63  , 151.04  , 150.51  ,
                              150.21  , 149.19  , 149.775 , 149.8957, 149.4532, 149.35  , 150.46  ,
                              150.    , 150.44  , 150.3999, 149.85  , 149.15  , 149.42  , 149.66  ,
                              149.1697, 149.15  , 148.4   , 148.01  , 148.2   , 148.45  , 147.93  ,
                              148.21  , 148.65  , 148.35  , 148.38  , 147.735 , 148.48  , 148.42  ,
                              148.42  , 148.38  , 149.08  , 148.81  , 148.8   , 148.7501, 148.76  ,
                              148.81  , 148.65  , 148.6   , 148.51  , 148.25  , 148.38  , 148.54  ,
                              148.6   , 150.6   , 150.21  , 150.06  , 149.16  , 149.4   , 148.85  ,
                              149.4   , 149.03  , 148.81  , 149.18  , 148.96  , 149.79  , 149.54  ,
                              149.47  , 149.    , 149.3   , 148.63  , 148.43  , 147.54  , 147.63  ,
                              147.5001, 147.87  , 148.75  , 150.74  , 150.8899, 150.85  , 150.99  ,
                              151.5611, 152.0754, 152.56  , 152.6741, 152.61  , 152.48  , 152.24  ,
                              152.29  , 151.8   , 151.595 , 151.24  , 151.79  , 151.9044, 152.03  ,
                              152.23  , 152.1669, 152.1144, 152.14  , 151.03  , 151.235 , 150.46  ,
                              150.47  , 150.51  , 150.47  , 150.64  , 150.54  , 150.5326, 150.65  ,
                              150.61  , 150.56  , 150.77  , 150.83  , 150.94  , 150.9   , 150.66  ,
                              150.84  , 150.9   ]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([150.4   , 150.4   , 150.91  , 150.64  , 150.65  , 150.95  , 150.71  ,
                             150.7   , 150.5   , 149.91  , 150.29  , 150.27  , 150.6   , 150.7   ,
                             150.5   , 150.81  , 150.89  , 150.79  , 152.    , 151.68  , 151.5   ,
                             150.74  , 150.51  , 149.9589, 150.49  , 150.    , 150.02  , 150.5199,
                             150.62  , 150.53  , 150.94  , 150.4811, 150.05  , 149.43  , 150.    ,
                             149.72  , 149.4   , 149.14  , 148.42  , 148.26  , 148.84  , 148.51  ,
                             148.76  , 149.    , 148.77  , 148.64  , 148.44  , 148.66  , 148.6   ,
                             148.5   , 148.5   , 149.16  , 149.04  , 148.87  , 148.82  , 148.79  ,
                             148.81  , 148.75  , 148.68  , 148.54  , 148.5   , 148.47  , 148.58  ,
                             148.6   , 150.98  , 150.64  , 150.2   , 150.03  , 149.4   , 149.44  ,
                             149.48  , 149.9   , 149.49  , 149.3   , 149.6   , 150.    , 150.1   ,
                             149.61  , 149.74  , 149.52  , 150.95  , 149.08  , 148.57  , 147.63  ,
                             148.01  , 148.0371, 150.1   , 151.14  , 151.47  , 151.255 , 151.87  ,
                             151.75  , 152.55  , 152.66  , 152.92  , 153.14  , 153.1   , 152.78  ,
                             152.5846, 152.5   , 151.86  , 151.9   , 151.93  , 152.04  , 152.37  ,
                             152.54  , 152.7   , 152.25  , 152.35  , 152.26  , 151.62  , 151.3595,
                             150.69  , 150.7   , 150.64  , 150.67  , 150.59  , 150.61  , 150.65  ,
                             150.67  , 150.72  , 150.77  , 150.92  , 150.94  , 150.96  , 150.93  ,
                             150.87  , 150.9   ]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([149.77  , 150.38  , 150.41  , 150.47  , 150.58  , 150.6   , 150.6   ,
                            150.51  , 149.55  , 149.01  , 149.84  , 150.    , 150.

In [79]:
def custom_indicator(close, high, low, start):
    #close = data['Close']
    #high = data['High']
    #low = data['Low']
    #open = data['Open']
    absolute_movement = high-low
    fib = absolute_movement*.32
    lowerbound = high-fib

    is_hammer = np.where((start > lowerbound) & (start - close > 0), 1, 0)
    return is_hammer

    #if open > lowerbound and open-close > 0:
    #    return close.index[-1]
    #else:
    #    return None

hammer_factory = vbt.IndicatorFactory(
    class_name = "Combination",
    short_name = "comb",
    input_names = ["close", "high", "low", "open"],
    output_names = ["value"],
    param_names=[],
).from_apply_func(
    custom_indicator
)

hammer = hammer_factory.run(data['Close'].T, data['High'].T, data['Low'].T, data['Open'].T)

hammer_true_dates = hammer[hammer['is_hammer'] == True].index
print(hammer_true_dates)

#print(hammer.value)

#hammer = talib.CDLHAMMER(data['Open'], data['High'], data['Low'], #data['Close'])
#print(hammer[hammer==100])





KeyError: 'is_hammer'